In [ ]:
!pip install datasets
from tqdm import tqdm
from datasets import list_datasets, load_dataset
import pickle
from google.colab import drive
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import random
import numpy as np
################################################################################
drive.mount('/content/drive', force_remount=True)
!cp /content/drive/MyDrive/NLP/UITws_v1.py /content
from UITws_v1 import UITws_v1
################################################################################
class MyDataset(Dataset):
  def __init__(self, x, y):
    self.data = x
    self.labels = y

  def __len__(self):
    return len(self.labels)
  
  def __getitem__(self, index):
    return self.data[index], self.labels[index]
################################################################################
class Accuracy:
    """A class to keep track of the accuracy while training"""
    def __init__(self):
        self.correct = 0
        self.total = 0
        
    def reset(self):
        """Resets the internal state"""
        self.correct = 0
        self.total = 0
        
    def update(self, outputs, labels):
        """
        Updates the internal state to later compute the overall accuracy
        
        output: the output of the network for a batch
        labels: the target labels
        """
        _, predicted = torch.max(outputs.data, 1) # predicted now contains the predicted class index/label
        
        self.total += labels.size(0)
        self.correct += (predicted == labels).sum().item() # .item() gets the number, not the tensor
        #self.correct += ((outputs.data > 0.5) == labels).sum().item()

    def compute(self):
        return self.correct/self.total
################################################################################
def test_accu():
    net.eval()
    test_accuracy = Accuracy()
    test_accuracy.reset()        
    with torch.no_grad():
        for test_data in test_loader:
            # get the data points
            test_inputs, test_labels = test_data
            test_inputs, test_labels = test_inputs.to(device).long(), test_labels.to(device)
            # forward the data through the network
            test_outputs = net(test_inputs)
            
            test_accuracy.update(test_outputs, test_labels)
            
    print("\nTesting Accuracy: {:.2f}%".format(100 * test_accuracy.compute()))
################################################################################
dataset = load_dataset('uit-nlp/vietnamese_students_feedback')
word_seg = UITws_v1('/content/drive/MyDrive/NLP/base_sep_sfx.pkl')
################################################################################
!pip install transformers
from transformers import AutoModel, AutoTokenizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 431 kB 36.8 MB/s 
     |████████████████████████████████| 115 kB 71.0 MB/s 
     |████████████████████████████████| 163 kB 66.5 MB/s 
     |████████████████████████████████| 212 kB 53.0 MB/s 
     |████████████████████████████████| 127 kB 75.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Mounted at /content/drive


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset vietnamese_students_feedback downloaded and prepared to /root/.cache/huggingface/datasets/uit-nlp___vietnamese_students_feedback/default/1.0.0/b01aa0fea00f1179ef9c093babb5e91f44bc60f19ded600754a08e018503e40f. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 36.3 MB/s 
     |████████████████████████████████| 6.6 MB 61.3 MB/s 


In [ ]:
train_dataset = dataset['train']
test_dataset = dataset['test']
#train_dataset['sentence'][0]
#train_dataset['sentiment'][0]

In [ ]:
segmented_texts = word_seg.segment(texts=train_dataset['sentence'], pre_tokenized=True, batch_size=256)
segmented_texts2 = word_seg.segment(texts=test_dataset['sentence'], pre_tokenized=True, batch_size=256)

In [ ]:
word_tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-word")
bartpho_word = AutoModel.from_pretrained("vinai/bartpho-word")

Downloading:   0%|          | 0.00/897 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/895k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

In [ ]:
#TXT = ['Chúng_tôi là những nghiên_cứu_viên .','Chúng_tôi là những nghiên_cứu_viên .'] 
input_ids = word_tokenizer(segmented_texts, return_tensors='pt', padding=True)['input_ids']

In [ ]:
input_ids2 = word_tokenizer(segmented_texts2, return_tensors='pt', padding=True)['input_ids']

In [ ]:
input_ids.size()

torch.Size([11426, 129])

In [ ]:
train_data = MyDataset(input_ids, train_dataset['sentiment'])

In [ ]:
test_data = MyDataset(input_ids2, test_dataset['sentiment'])

In [ ]:
features[0].size()

torch.Size([2, 7, 1024])

In [ ]:
features.last_hidden_state == features[0]

tensor([[[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]]])

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True, num_workers=2)

In [ ]:
vector_size = 1024

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
bartpho_word = bartpho_word.to(device)
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1,
                                out_channels = 100,
                                kernel_size = (2,vector_size))
        self.conv2 = nn.Conv2d(in_channels = 1,
                                out_channels = 100,
                                kernel_size = (3,vector_size))
        self.conv3 = nn.Conv2d(in_channels = 1,
                                out_channels = 100,
                                kernel_size = (4,vector_size))
        self.conv4 = nn.Conv2d(in_channels = 1,
                                out_channels = 100,
                                kernel_size = (5,vector_size))
        self.fc1 = nn.Linear(400, 3)
        self.dropout = nn.Dropout(0.30)
        self.dropout1 = nn.Dropout(0.30)
        self.dropout2 = nn.Dropout(0.30)
        self.dropout3 = nn.Dropout(0.30)
        self.dropout4 = nn.Dropout(0.30)
        self.dropout5 = nn.Dropout(0.30)
    def forward(self, x):
        with torch.no_grad():
            x = bartpho_word(x).last_hidden_state.unsqueeze(1)
        x = self.dropout(x)
        x1 = F.relu(self.conv1(x))
        x1 = self.dropout1(F.max_pool2d(x1, (x1.size()[-2],1)))
        x2 = F.relu(self.conv2(x))
        x2 = self.dropout2(F.max_pool2d(x2, (x2.size()[-2],1)))
        x3 = F.relu(self.conv3(x))
        x3 = self.dropout3(F.max_pool2d(x3, (x3.size()[-2],1)))
        x4 = F.relu(self.conv4(x))
        x4 = self.dropout4(F.max_pool2d(x4, (x4.size()[-2],1)))
        x = torch.flatten(torch.cat((x1,x2,x3,x4),-3),start_dim=-3)
        x = self.fc1(self.dropout5(x))
        return x

net = Net()
net = net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
#optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.5)

In [ ]:
#net = net.float()
net.train()
live_accuracy = Accuracy()

for epoch in range(100):  # loop over the dataset multiple times
    print("\nStarting epoch {}".format(epoch+1))

    live_accuracy.reset()
    total = 0
    running_loss = 0.0

    # to make a beautiful progress bar
    loader = tqdm(enumerate(train_loader), total=len(train_loader))
    for i, data in loader:
        # get the data points
        inputs, labels = data
        inputs, labels = inputs.to(device).long(), labels.to(device)
        # zero the parameter gradients (else, they are accumulated)
        optimizer.zero_grad()

        # forward the data through the network
        outputs = net(inputs)
        # calculate the loss given the output of the network and the target labels
        loss = criterion(outputs, labels)
        # calculate the gradients of the network w.r.t. its parameters
        loss.backward()
        # Let the optimiser take an optimization step using the calculated gradients
        optimizer.step()
        
        running_loss += loss
        total += outputs.size(0)

        live_accuracy.update(outputs, labels)
        loader.set_description("loss: {:.5f}|acc: {:.2f}%".format(running_loss/total,100 * live_accuracy.compute()))
    test_accu()
    net.train()

print('Finished Training')


Starting epoch 1


loss: 0.00262|acc: 94.24%: 100%|██████████| 179/179 [05:38<00:00,  1.89s/it]



Testing Accuracy: 91.72%

Starting epoch 2


loss: 0.00246|acc: 94.88%: 100%|██████████| 179/179 [05:39<00:00,  1.89s/it]



Testing Accuracy: 91.31%

Starting epoch 3


loss: 0.00218|acc: 95.67%:   7%|▋         | 13/179 [00:26<05:37,  2.03s/it]


KeyboardInterrupt: ignored

In [ ]:
inputs.size()

torch.Size([5, 129])

In [ ]:
bartpho_word(inputs).last_hidden_state.unsqueeze(1).is_cuda

True

In [ ]:
pickle.dump(net,open('/content/drive/MyDrive/NLP/imdb_sentiment.p','wb'))

In [ ]:
net = pickle.load(open('/content/drive/MyDrive/NLP/imdb_sentiment.p','rb'))

In [ ]:
net.eval()
accuracy = Accuracy()
accuracy.reset()
# Gradients are calculated on the forward pass for every iteration.
# As we do not need gradients now, we can disable the calculation.
with torch.no_grad():
    for data in tqdm(train_loader):
        # get the data points
        inputs, labels = data
        inputs, labels = inputs.to(device).long(), labels.to(device)
        # forward the data through the network
        outputs = net(inputs)
        
        accuracy.update(outputs, labels)

print("Training Accuracy: {:.2f}%".format(100 * accuracy.compute()))

100%|██████████| 500/500 [00:08<00:00, 57.54it/s]

Training Accuracy: 92.60%


In [ ]:
net.eval()
accuracy = Accuracy()
accuracy.reset()        
with torch.no_grad():
    for data in tqdm(val_loader):
        # get the data points
        inputs, labels = data
        inputs, labels = inputs.to(device).long(), labels.to(device)
        # forward the data through the network
        outputs = net(inputs)
        
        accuracy.update(outputs, labels)
        
print("\nTesting Accuracy: {:.2f}%".format(100 * accuracy.compute()))

100%|██████████| 500/500 [00:08<00:00, 58.67it/s]


Testing Accuracy: 88.30%
